# Predict which waterpumps in Tanzania are faulty

## Get Data and look at it

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

pd.set_option('display.max_columns', None)  # Unlimited columns
pd.options.mode.use_inf_as_na = True

In [105]:
!pip install -U seaborn

Requirement already up-to-date: seaborn in /usr/local/lib/python3.6/dist-packages (0.9.0)


In [106]:
sns.__version__

'0.9.0'

In [107]:
X_train_original = pd.read_csv('./train_features.csv', header = [0], index_col = 0)
X_train_original.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [108]:
y_train_original = pd.read_csv('./train_labels.csv', header = [0], index_col = 0)
y_train_original.head()

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


In [109]:
pd.value_counts(y_train_original.status_group, normalize = True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

# Majority class prediction
### As a first step, let's see if we get around 50% accuracy on the test set, with a majority class prediction.

In [110]:
X_test_original = pd.read_csv('./test_features.csv', header = [0], index_col = 0)
X_test_original.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [111]:
X_test_original.shape

(14358, 39)

In [112]:
y_pred = ['functional'] * len(X_test_original)
y_pred = pd.DataFrame(data = y_pred, 
                      index = X_test_original.index.values,
                      columns = ['status_group'])
y_pred.head()

,status_group
50785,functional
51630,functional
17168,functional
45559,functional
49871,functional


In [0]:
y_pred.to_csv('./majority_class_pred.csv', 
          header = ['status_group'],
          index = True,
             index_label = 'id')

In [114]:
y_pred.shape

(14358, 1)

### This gives us a score of 0.53

# Predict with just the numerical features

In [115]:
# numerical = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
X_train_numerical = X_train_original.select_dtypes(include = np.number).copy()
X_train_numerical.head()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
id,,,,,,,,,
69572,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999
8776,0.0,1399,34.698766,-2.147466,0,20,2,280,2010
34310,25.0,686,37.460664,-3.821329,0,21,4,250,2009
67743,0.0,263,38.486161,-11.155298,0,90,63,58,1986
19728,0.0,0,31.130847,-1.825359,0,18,1,0,0


### Convert date_recorded to days since epoch

In [116]:
days_since_epoch = pd.to_datetime(X_train_original['date_recorded']) - pd.datetime(1970, 1, 1)
X_train_numerical['days_since_epoch'] = days_since_epoch.dt.days
X_train_numerical.head()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,days_since_epoch
id,,,,,,,,,,
69572,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999,15047
8776,0.0,1399,34.698766,-2.147466,0,20,2,280,2010,15770
34310,25.0,686,37.460664,-3.821329,0,21,4,250,2009,15761
67743,0.0,263,38.486161,-11.155298,0,90,63,58,1986,15733
19728,0.0,0,31.130847,-1.825359,0,18,1,0,0,15168


In [117]:
X_train_numerical_indices = X_train_numerical.index.values
y_train_numerical = \
  y_train_original[y_train_original.index.isin(X_train_numerical_indices)]
y_train_numerical.head()

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


### Check X_train_numerical and y_train_numerical

In [118]:
X_train_numerical.shape, y_train_numerical.shape

((59400, 10), (59400, 1))

In [119]:
np.all(y_train_numerical.index.values == \
       X_train_numerical.index.values)

True

### Let's try with a LogisticRegression Classifier

In [120]:
cv_score = cross_val_score(LogisticRegression(), 
                            X_train_numerical, y_train_numerical,
                            scoring = 'accuracy',
                            cv = 3,
                            n_jobs = -1,
                            verbose = 1)
cv_score

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    4.2s finished


array([0.55909091, 0.55666667, 0.55712121])

### LogisticRegression classifier gave us a score of 0.55

### Let's try with a DecisionTreeClassifier

In [121]:
clf = DecisionTreeClassifier()
cv_score = cross_val_score(clf, 
                            X_train_numerical, y_train_numerical,
                            scoring = 'accuracy',
                            cv = 3,
                            n_jobs = -1,
                            verbose = 1)
cv_score

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.2s finished


array([0.67419192, 0.66929293, 0.66121212])

### DecisionTree classifier gave us a score of 0.65.  Let's write it out to a file and submit it.

In [0]:
clf.fit(X_train_numerical, y_train_numerical)
X_test_numerical = X_test_original.select_dtypes(include = np.number).copy()
days_since_epoch = pd.to_datetime(X_test_original['date_recorded']) - \
                   pd.datetime(1970, 1, 1)
X_test_numerical['days_since_epoch'] = days_since_epoch.dt.days

y_pred = clf.predict(X_test_numerical)

In [123]:
y_pred = pd.DataFrame(data = y_pred, 
                      index = X_test_numerical.index.values,
                      columns = ['status_group'])
y_pred.head()

,status_group
50785,non functional
51630,functional
17168,functional
45559,non functional
49871,functional


In [0]:
y_pred.to_csv('./decision_tree_pred.csv', 
          header = ['status_group'],
          index = True,
             index_label = 'id')

## Check data for missing or unusual values

In [125]:
X_train_original.isnull().sum()

amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity

In [126]:
X_train_original.shape

(59400, 39)

### Let's drop columns that have nulls

In [127]:
X_non_nulls = X_train_original.dropna(axis = 1)
print('X_non_nulls.shape:', X_non_nulls.shape)
X_non_nulls.head()

X_non_nulls.shape: (59400, 32)


,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,recorded_by,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,1390,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,GeoData Consultants Ltd,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,1399,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,GeoData Consultants Ltd,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,686,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,GeoData Consultants Ltd,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,263,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,GeoData Consultants Ltd,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,0,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,GeoData Consultants Ltd,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


### How many unique values does each column have?

In [128]:
X_non_nulls.nunique().sort_values(ascending = True)

recorded_by                  1
source_class                 3
management_group             5
quantity_group               5
quantity                     5
quality_group                6
waterpoint_type_group        6
payment_type                 7
payment                      7
source_type                  7
waterpoint_type              7
extraction_type_class        7
water_quality                8
basin                        9
source                      10
management                  12
extraction_type_group       13
extraction_type             18
district_code               20
region                      21
region_code                 27
construction_year           55
num_private                 65
amount_tsh                  98
lga                        125
date_recorded              356
population                1049
ward                      2092
gps_height                2428
wpt_name                 37400
longitude                57516
latitude                 57517
dtype: i

### According to [this article](https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931), the Decision Tree classifier is faster when categorical values are encoded numeric or binary.
### Let's encode non-null columns that have < 50 unique values, add the numerical columns to that dataframe, and run a Decision Tree classifier with various depths.

In [129]:
X_selected = X_non_nulls.loc[:, X_non_nulls.nunique().sort_values() < 50]
print('X_selected.shape:', X_selected.shape)
X_selected.head()

X_selected.shape: (59400, 21)


,basin,region,region_code,district_code,recorded_by,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,Lake Nyasa,Iringa,11,5,GeoData Consultants Ltd,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,Lake Victoria,Mara,20,2,GeoData Consultants Ltd,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,Pangani,Manyara,21,4,GeoData Consultants Ltd,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,Ruvuma / Southern Coast,Mtwara,90,63,GeoData Consultants Ltd,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,Lake Victoria,Kagera,18,1,GeoData Consultants Ltd,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [130]:
cat_cols = list(X_selected.select_dtypes(['object']).columns.values)

X_categorical = X_selected[cat_cols]. \
                  apply(lambda x: x.astype('category').cat.codes)
X_categorical.head()


,basin,region,recorded_by,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,
69572,1,3,0,3,1,0,7,4,2,0,6,2,1,1,8,6,0,1,1
8776,4,9,0,3,1,0,11,4,0,2,6,2,2,2,5,3,1,1,1
34310,5,8,0,3,1,0,7,4,4,5,6,2,1,1,0,1,1,2,1
67743,7,12,0,14,10,5,7,4,0,2,6,2,0,0,3,0,0,2,1
19728,4,4,0,3,1,0,1,1,0,2,6,2,3,3,5,3,1,1,1


In [131]:
X_train_selected = X_train_numerical.join(X_categorical)
X_train_selected.shape

(59400, 29)

In [132]:
clf = DecisionTreeClassifier()
cv_score = cross_val_score(clf, 
                            X_train_selected, y_train_original,
                            scoring = 'accuracy',
                            cv = 3,
                            n_jobs = -1,
                            verbose = 1)
cv_score

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.9s finished


array([0.75565657, 0.74974747, 0.74722222])

### Gives us a score of 0.75

### Let's apply the classifier to the test data

In [0]:
clf.fit(X_train_selected, y_train_original)

X_test_non_nulls = X_test_original.dropna(axis = 1)
X_test_selected = X_test_non_nulls.loc[:, X_test_non_nulls.nunique().sort_values() < 50]

cat_cols = list(X_test_selected.select_dtypes(['object']).columns.values)
X_test_categorical = X_test_selected[cat_cols]. \
                        apply(lambda x: x.astype('category').cat.codes)

X_test_selected = X_test_numerical.join(X_test_categorical)

In [134]:
y_pred = clf.predict(X_test_selected)
y_pred = pd.DataFrame(data = y_pred, 
                      index = X_test_selected.index.values,
                      columns = ['status_group'])
y_pred.head()

,status_group
50785,non functional
51630,functional
17168,functional needs repair
45559,non functional
49871,functional


In [0]:
y_pred.to_csv('./decision_tree_pred_with_categorical.csv', 
          header = ['status_group'],
          index = True,
             index_label = 'id')

## Since our Decision Tree gave us a good result, let's try Random Forest classifier.
### Random Forest classifiers are good for multinomial targets. This classifier takes random samples from the training dataset, so there is no need to do cross validation on it.  We may do GridSearchCV to try different n_estimators and max_depth (if our score is not very good).

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    X_train_selected, y_train_original, test_size=0.2)

In [137]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.798989898989899

### Gives us a score of 0.79. Better, but not by much

## Let's try RandomForest with Grid Search

In [138]:
# If you're having problems getting the right keys
# in the param_grid structure, this function will
# return the list of keys that go in there:
#
# RandomForestClassifier().get_params().keys()

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [6, 10, 20, 30]
}

gridsearch = GridSearchCV(RandomForestClassifier(n_jobs = -1), 
                          param_grid=param_grid, 
                          scoring='accuracy', cv=3, 
                          return_train_score=True, verbose=10)

gridsearch.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] max_depth=6, n_estimators=10 ....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  max_depth=6, n_estimators=10, score=0.7195075757575757, total=   0.5s
[CV] max_depth=6, n_estimators=10 ....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  max_depth=6, n_estimators=10, score=0.720770202020202, total=   0.4s
[CV] max_depth=6, n_estimators=10 ....................................


...

[CV]  max_depth=30, n_estimators=30, score=0.7944444444444444, total=   1.9s
[CV] max_depth=30, n_estimators=30 ...................................


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  max_depth=30, n_estimators=30, score=0.7921717171717172, total=   2.1s


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:   41.7s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 20, 30], 'max_depth': [6, 10, 20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=10)

In [139]:
pd.DataFrame(gridsearch.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
8,1.559696,0.118298,0.798990,0.949716,20,30,"{'max_depth': 20, 'n_estimators': 30}",1,0.802146,0.949653,0.798359,0.948390,0.796465,0.951105,0.011578,0.000228,0.002362,0.001109
7,1.034794,0.118648,0.794844,0.947917,20,20,"{'max_depth': 20, 'n_estimators': 20}",2,0.796402,0.946875,0.794003,0.944792,0.794129,0.952083,0.036792,0.000476,0.001102,0.003067
11,1.789384,0.153083,0.794045,0.994592,30,30,"{'max_depth': 30, 'n_estimators': 30}",3,0.795518,0.994571,0.794444,0.994823,0.792172,0.994381,0.056964,0.047973,0.001395,0.000181
10,1.124861,0.118634,0.793918,0.991235,30,20,"{'max_depth': 30, 'n_estimators': 20}",4,0.795896,0.991446,0.793119,0.991446,0.792740,0.990814,0.006700,0.000329,0.001407,0.000298
6,0.589411,0.118477,0.788742,0.939867,20,10,"{'max_depth': 20, 'n_estimators': 10}",5,0.786932,0.942614,0.791919,0.939457,0.787374,0.937532,0.008141,0.000055,0.002254,0.002095
9,0.641235,0.119074,0.788258,0.980450,30,10,"{'max_depth': 30, 'n_estimators': 10}",6,0.789331,0.980208,0.788321,0.980966,0.787121,0.980177,0.004660,0.000491,0.000903,0.000365
3,0.430171,0.118474,0.753325,0.771433,10,10,"{'max_depth': 10, 'n_estimators': 10}",7,0.752273,0.771086,0.755871,0.771749,0.751831,0.771465,0.031549,0.000257,0.001810,0.000272
4,0.706189,0.118290,0.752715,0.770360,10,20,"{'max_depth': 10, 'n_estimators': 20}",8,0.751452,0.771875,0.757702,0.771528,0.748990,0.767677,0.019487,0.000106,0.003667,0.001903
5,0.982046,0.118389,0.750947,0.767856,10,30,"{'max_depth': 10, 'n_estimators': 30}",9,0.750694,0.770297,0.754356,0.767929,0.747790,0.765341,0.007823,0.000323,0.002686,0.002024
2,0.718732,0.118253,0.721843,0.723316,6,30,"{'max_depth': 6, 'n_estimators': 30}",10,0.722664,0.724653,0.722980,0.722128,0.719886,0.723169,0.025092,0.000165,0.001390,0.001036


### Looks like max_depth of 20 and n_estimators of 30 gives the best fit. So let's build a classifier with those parameters and verify that it works with X_train/y_train.

In [140]:
clf = RandomForestClassifier(max_depth = 20, n_estimators = 30, n_jobs = -1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.8104377104377104

### Gives us a score of 0.81

### Get the prediction y_pred and submit it

In [141]:
y_pred = clf.predict(X_test_selected)
y_pred = pd.DataFrame(data = y_pred, 
                      index = X_test_selected.index.values,
                      columns = ['status_group'])
y_pred.head()

,status_group
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional


In [0]:
y_pred.to_csv('./random_forest_pred.csv', 
          header = ['status_group'],
          index = True,
             index_label = 'id')